# When / Otherwise

- This requirement is similar to the last, but now you want to add multiple values based on the voter's position. Modify your `voter_df` DataFrame to add a random number to any voting member that is defined as a `Councilmember`. Use `2` for the `Mayor` and `0` for anything other position.

- The `voter_df` Data Frame is defined and available to you. The `pyspark.sql.functions` library is available as `F`. You can use `F.rand()` to generate the random value.

## Instructions

- Add a column to `voter_df` named `random_val` with the results of the `F.rand()` method for any voter with the title `Councilmember`. Set `random_val` to `2` for the `Mayor`. Set any other title to the value `0`.
- Show some of the Data Frame rows, noting whether the clauses worked.
- Use the `.filter` clause to find 0 in `random_val`.

In [6]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [7]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [8]:
!pwd

/home/talentum/test-jupyter/P3/M2/SM2/2_ConditionalDataFramecolumnoperations


In [16]:
import pyspark.sql.functions as F

# Load the CSV file
voter_df = spark.read.format('csv').options(Header=True).load('file:///home/talentum/test-jupyter/P3/M2/SM2/2_ConditionalDataFramecolumnoperations/Dataset/DallasCouncilVoters.csv.gz')

# Add a column to voter_df for a voter based on their position
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE == 'Councilmember', 1)
                              .when(voter_df.TITLE == 'Mayor', 2)
                              .otherwise(F.rand()))

# Show some of the DataFrame rows
voter_df.show()

# Use the .filter() clause with random_val
voter_df.filter(voter_df.random_val > 0).show()
                               

+----------+-------------+-------------------+----------+
|      DATE|        TITLE|         VOTER_NAME|random_val|
+----------+-------------+-------------------+----------+
|02/08/2017|Councilmember|  Jennifer S. Gates|       1.0|
|02/08/2017|Councilmember| Philip T. Kingston|       1.0|
|02/08/2017|        Mayor|Michael S. Rawlings|       2.0|
|02/08/2017|Councilmember|       Adam Medrano|       1.0|
|02/08/2017|Councilmember|       Casey Thomas|       1.0|
|02/08/2017|Councilmember|Carolyn King Arnold|       1.0|
|02/08/2017|Councilmember|       Scott Griggs|       1.0|
|02/08/2017|Councilmember|   B. Adam  McGough|       1.0|
|02/08/2017|Councilmember|       Lee Kleinman|       1.0|
|02/08/2017|Councilmember|      Sandy Greyson|       1.0|
|02/08/2017|Councilmember|  Jennifer S. Gates|       1.0|
|02/08/2017|Councilmember| Philip T. Kingston|       1.0|
|02/08/2017|        Mayor|Michael S. Rawlings|       2.0|
|02/08/2017|Councilmember|       Adam Medrano|       1.0|
|02/08/2017|Co

In [11]:
voter_df.select('TITLE').distinct().show()

+--------------------+
|               TITLE|
+--------------------+
|               Mayor|
|       Councilmember|
|Deputy Mayor Pro Tem|
|                null|
|              2020]"|
| authorize an  or...|
| authorize an  or...|
|              Vacant|
| authorize an  or...|
|   authorize   an...|
|       Mayor Pro Tem|
+--------------------+



In [23]:
voter_df.withColumn('First_Name', F.split(voter_df.VOTER_NAME, '\s+').getItem(0)) \
    .withColumn('Last_Name', F.split(voter_df.VOTER_NAME, '\s+').getItem(F.size(F.split(voter_df.VOTER_NAME, '\s+')) - 1)) \
    .drop('VOTER_NAME') \
    .show()


+----------+-------------+----------+----------+---------+
|      DATE|        TITLE|random_val|First_Name|Last_Name|
+----------+-------------+----------+----------+---------+
|02/08/2017|Councilmember|       1.0|  Jennifer|    Gates|
|02/08/2017|Councilmember|       1.0|    Philip| Kingston|
|02/08/2017|        Mayor|       2.0|   Michael| Rawlings|
|02/08/2017|Councilmember|       1.0|      Adam|  Medrano|
|02/08/2017|Councilmember|       1.0|     Casey|   Thomas|
|02/08/2017|Councilmember|       1.0|   Carolyn|   Arnold|
|02/08/2017|Councilmember|       1.0|     Scott|   Griggs|
|02/08/2017|Councilmember|       1.0|        B.|  McGough|
|02/08/2017|Councilmember|       1.0|       Lee| Kleinman|
|02/08/2017|Councilmember|       1.0|     Sandy|  Greyson|
|02/08/2017|Councilmember|       1.0|  Jennifer|    Gates|
|02/08/2017|Councilmember|       1.0|    Philip| Kingston|
|02/08/2017|        Mayor|       2.0|   Michael| Rawlings|
|02/08/2017|Councilmember|       1.0|      Adam|  Medran